# Original Dataset - ML Test

In [1]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import regularizers

import sqlalchemy
from sqlalchemy import create_engine, inspect

import pandas as pd
import tensorflow as tf
import keras_tuner as kt
from pprint import pprint

import os
import time
from datetime import datetime

%run functions.ipynb

In [2]:
# Time the run
start_time = time.time()

In [3]:
# Check for untitled_project folder
if os.path.exists("untitled_project"):
    input("DELETE untitled_project")

## Import datasets

In [4]:
# Import the data
engine = create_engine("sqlite:///voice.sqlite")

# View all of the classes
inspector = inspect(engine)
table_names = inspector.get_table_names()
table_names

['aval',
 'bval',
 'demographic',
 'diagnosis',
 'gval',
 'habits',
 'rval',
 'spectrogram']

In [5]:
# Initialise a dictionary to hold dataframes
dataframes = dict()

# Loop through each table
for table in table_names:
    
    # Dataframe name
    df_name = f'{table}_df'
    
    # Create dataframe
    dataframes[df_name] = pd.read_sql(
        f'SELECT * FROM {table}',
        engine
    )

In [6]:
# Merge dataframes
merged_df = pd.merge(
    dataframes['demographic_df'],
    dataframes['diagnosis_df'],
    how = 'inner',
    on = 'id'
)

merged_df = pd.merge(
    merged_df,
    dataframes['habits_df'],
    how = 'inner',
    on = 'id'
)

# Display merged_df
merged_df.head()

,id,age,gender,occupation_status,diagnosis,subtype,vhi_score,rsi_score,alcohol_consumption,alcohol_pd,...,chocolate,chocolate_grams_pd,coffee,coffee_pd,citrus_fruits,citrus_fruits_pd,soft_cheese,soft_cheese_pd,tomatoes,water_litres_pd
0,voice100,24,m,unknown,healthy,no subtype,0,5,casual,0.36,...,sometimes,30,always,3,never,0.00,almost always,100,never,1.5
1,voice101,60,m,unknown,healthy,no subtype,80,10,nondrinker,0.00,...,sometimes,30,always,4,never,0.00,sometimes,100,sometimes,1.5
2,voice192,22,m,cook,hyperkinetic dysphonia,no subtype,0,10,nondrinker,0.00,...,always,14,always,3,almost always,1.17,sometimes,100,sometimes,2.5
3,voice193,46,f,housewife,hyperkinetic dysphonia,no subtype,0,36,casual,0.36,...,sometimes,30,always,2,sometimes,1.00,sometimes,100,sometimes,1.0
4,voice008,51,f,researcher,reflux laryngitis,no subtype,19,15,casual,0.36,...,almost always,20,always,2,almost always,1.00,sometimes,100,almost always,1.0


## Preprocessing

### Separate the target and feature variables

In [7]:
# Drop the 'id' column
no_id_df = merged_df.drop(columns=['id'])

In [8]:
# Define the target variables
target_var = ['diagnosis', 'subtype']
y = no_id_df[target_var]

# Define the feature variables
X = no_id_df.drop(columns=target_var)

### Binary classification - `diagnosis`

In [9]:
# Encode the target variable, ignore subtype
y = y['diagnosis'].apply(encode_binary)
y

0      0
1      0
2      1
3      1
4      1
      ..
199    0
200    1
201    1
202    0
203    0
Name: diagnosis, Length: 204, dtype: int64

### Bin `occupation_status` column

In [10]:
# Use limit_unique() function to bin the column
limit_unique(X, 10, ['occupation_status'])

occupation_status
unknown               41
researcher            41
other                 25
employee              25
housewife             23
student               16
technical operator    12
singer                10
pensioner              6
doctor                 5
Name: count, dtype: int64
Number of unique values: 10



### Encode feature columns

#### Encoding
- `smoker` column
	- `0` for `no`
	- `1` for `casual`
	- `2` for `yes`
- `alcohol_consumption` column
	- `0` for `nondrinker`
	- `1` for `casual`
	- `2` for `habitual`
- `carbonated_beverages`, `tomatoes`, `coffee`, `chocolate`, `soft_cheese`, `citrus_fruits` columns
	- `0` for `never`
	- `1` for `almost never`
	- `2` for `sometimes`
	- `3` for `almost always`
	- `4` for `always`

In [11]:
# Define the maps
smoker_map = {
    'no': 0,
    'casual': 1,
    'yes': 2
}

alcohol_map = {
    'nondrinker': 0,
    'casual': 1,
    'habitual': 2
}

habit_map = {
    'never': 0,
    'almost never': 1,
    'sometimes': 2,
    'almost always': 3,
    'always': 4
}

In [12]:
# Apply the label encoding using the maps
X['smoker'] = X['smoker'].map(smoker_map)
X['alcohol_consumption'] = X['alcohol_consumption'].map(alcohol_map)

# Habit columns
habit_cols = [
    'carbonated_beverages', 'tomatoes',
    'coffee', 'chocolate',
    'soft_cheese', 'citrus_fruits'
]

# Use a loop for the habit columns
for habit in habit_cols:
    X[habit] = X[habit].map(habit_map)

In [13]:
# Remove the occupation_status column
# X = X.drop(columns=['occupation_status'])

In [14]:
# Encode the categorical columns using get_dummies
categorical_hot = ['gender', 'occupation_status']
# categorical_hot = ['gender']

# One-hot encoding
encoded_columns = pd.get_dummies(X[categorical_hot]).astype(int)

# Update the feature dataframe
X.drop(categorical_hot, axis=1, inplace=True)
X = pd.concat([X, encoded_columns], axis=1)

In [15]:
# Display the dataframe
X.info()
X.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 31 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   age                                   204 non-null    int64  
 1   vhi_score                             204 non-null    int64  
 2   rsi_score                             204 non-null    int64  
 3   alcohol_consumption                   204 non-null    int64  
 4   alcohol_pd                            204 non-null    float64
 5   smoker                                204 non-null    int64  
 6   cigarettes_pd                         204 non-null    int64  
 7   carbonated_beverages                  204 non-null    int64  
 8   carbonated_pd                         204 non-null    float64
 9   chocolate                             204 non-null    int64  
 10  chocolate_grams_pd                    204 non-null    int64  
 11  coffee             

,age,vhi_score,rsi_score,alcohol_consumption,alcohol_pd,smoker,cigarettes_pd,carbonated_beverages,carbonated_pd,chocolate,...,occupation_status_doctor,occupation_status_employee,occupation_status_housewife,occupation_status_other,occupation_status_pensioner,occupation_status_researcher,occupation_status_singer,occupation_status_student,occupation_status_technical operator,occupation_status_unknown
0,24,0,5,1,0.36,0,0,3,3.00,2,...,0,0,0,0,0,0,0,0,0,1
1,60,80,10,0,0.00,0,0,3,3.00,2,...,0,0,0,0,0,0,0,0,0,1
2,22,0,10,0,0.00,0,0,0,0.00,4,...,0,0,0,1,0,0,0,0,0,0
3,46,0,36,1,0.36,2,15,2,0.61,2,...,0,0,1,0,0,0,0,0,0,0
4,51,19,15,1,0.36,0,0,1,0.09,3,...,0,0,0,0,0,1,0,0,0,0


### Split and Scale

In [16]:
# Split the preprocessed data to training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

In [17]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Hyperparameter Tuning

In [18]:
# Define the model parameters
number_input_features = len(X_train.columns)

# Maximum hidden layers (min. 2 for DL)
max_hidden_layers = 4

# Maximum neurons per hidden layer
max_num_neurons = number_input_features * 2 - 1

# Step count
step_count = 5

# Hidden layer activation functions
activation_functions = [
    'relu', 'leaky_relu', 'tanh',
    'elu', 'selu', 'exponential',
    'softmax', 'softplus'
]

# Define the output layer
output_layer_neurons = 1
output_layer_activation = "sigmoid"

# Model compilation
compile_loss = "binary_crossentropy"
compile_opt = "adam"

# Number of max epochs
tuner_max_epochs = 20
search_max_epochs = 20

# Hyperband iterations
hp_iterations = 2

# Regularizers
reg_kernel = regularizers.L1(0.01)

In [19]:
# Extract class name and parameter value, for performance tracker
class_name = reg_kernel.__class__.__name__

if class_name == "L1L2":
    value_l1 = reg_kernel.get_config()['l1']
    value_l2 = reg_kernel.get_config()['l2']

    # Create string version
    reg_kernel_string = f"{class_name}(l1={value_l1:.3f})(l2={value_l2:.3f})"

else:
    param_value = reg_kernel.get_config()[f'{class_name.lower()}']

    # Create string version
    reg_kernel_string = f"{class_name}({param_value:.3f})"

print(reg_kernel_string)

L1(0.010)


In [20]:
# Initialise the Hyperband tuner
tuner = kt.Hyperband(
    create_model,
    objective = "val_accuracy",
    max_epochs = tuner_max_epochs,
    hyperband_iterations = hp_iterations
)

In [21]:
# Find the best hyperparameters
tuner.search(
    X_train_scaled,
    y_train,
    epochs = search_max_epochs,
    validation_data = (X_test_scaled, y_test)
)

Trial 60 Complete [00h 00m 01s]
val_accuracy: 0.7254902124404907

Best val_accuracy So Far: 0.8823529481887817
Total elapsed time: 00h 00m 30s


## Compile, Train, Evaluate the Best Model

In [22]:
# Get the top 3 model hyperparameters
top3_hyper = tuner.get_best_hyperparameters(3)

for model in top3_hyper:
    pprint(model.values)

{'activation_layer_0': 'tanh',
 'activation_layer_1': 'leaky_relu',
 'activation_layer_2': 'leaky_relu',
 'activation_layer_3': 'tanh',
 'num_layers': 2,
 'tuner/bracket': 2,
 'tuner/epochs': 7,
 'tuner/initial_epoch': 3,
 'tuner/round': 1,
 'tuner/trial_id': '0002',
 'units_layer_0': 51,
 'units_layer_1': 61,
 'units_layer_2': 26,
 'units_layer_3': 51}
{'activation_layer_0': 'tanh',
 'activation_layer_1': 'leaky_relu',
 'activation_layer_2': 'leaky_relu',
 'activation_layer_3': 'tanh',
 'num_layers': 2,
 'tuner/bracket': 2,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/round': 2,
 'tuner/trial_id': '0012',
 'units_layer_0': 51,
 'units_layer_1': 61,
 'units_layer_2': 26,
 'units_layer_3': 51}
{'activation_layer_0': 'exponential',
 'activation_layer_1': 'relu',
 'activation_layer_2': 'elu',
 'activation_layer_3': 'softmax',
 'num_layers': 2,
 'tuner/bracket': 1,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/round': 1,
 'tuner/trial_id': '0051',
 'units_layer_0': 51,

In [23]:
# Get the top model
best_hyper = top3_hyper[0]
best_hyper.values

{'activation_layer_0': 'tanh',
 'units_layer_0': 51,
 'num_layers': 2,
 'units_layer_1': 61,
 'activation_layer_1': 'leaky_relu',
 'units_layer_2': 26,
 'activation_layer_2': 'leaky_relu',
 'tuner/epochs': 7,
 'tuner/initial_epoch': 3,
 'tuner/bracket': 2,
 'tuner/round': 1,
 'tuner/trial_id': '0002',
 'units_layer_3': 51,
 'activation_layer_3': 'tanh'}

In [24]:
# Define the model parameters
number_input_features = len(X_train.columns)
hidden_layer0_neurons = best_hyper.values['units_layer_0']
hidden_layer0_activation = best_hyper.values['activation_layer_0']

# Return the number of hidden layers
total_hidden = best_hyper.values['num_layers'] + 1

# Train model
model_train_epochs = 100

In [25]:
# Initialise the sequential model
nn = Sequential()

# Create the input layer and first hidden layer
nn.add(Dense(
    units = hidden_layer0_neurons,
    activation = hidden_layer0_activation,
    kernel_regularizer = reg_kernel,
    input_dim = number_input_features
))
        
# Create additional hidden layers
for layer in range(1, total_hidden):
    nn.add(Dense(
        units = best_hyper.values[f'units_layer_{layer}'],
        activation = best_hyper.values[f'activation_layer_{layer}'],
        kernel_regularizer = reg_kernel
    ))

# Create the output layer
nn.add(Dense(
    units = output_layer_neurons,
    activation = output_layer_activation
))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 51)                1632      
                                                                 
 dense_6 (Dense)             (None, 61)                3172      
                                                                 
 dense_7 (Dense)             (None, 26)                1612      
                                                                 
 dense_8 (Dense)             (None, 1)                 27        
                                                                 
Total params: 6443 (25.17 KB)
Trainable params: 6443 (25.17 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [26]:
# Compile the model
nn.compile(
    loss = compile_loss,
    optimizer = compile_opt,
    metrics = ["accuracy"]
)

In [27]:
# Train the model
fit_model = nn.fit(
    X_train_scaled,
    y_train,
    # shuffle = True,
    epochs = model_train_epochs,
    verbose = 1
)

Epoch 1/100
5/5 [==============================] - 0s 961us/step - loss: 8.4404 - accuracy: 0.4575
Epoch 2/100
5/5 [==============================] - 0s 960us/step - loss: 8.1379 - accuracy: 0.5752
Epoch 3/100
5/5 [==============================] - 0s 950us/step - loss: 7.8493 - accuracy: 0.6797
Epoch 4/100
5/5 [==============================] - 0s 926us/step - loss: 7.5720 - accuracy: 0.7582
Epoch 5/100
5/5 [==============================] - 0s 1ms/step - loss: 7.3016 - accuracy: 0.7712
Epoch 6/100
5/5 [==============================] - 0s 938us/step - loss: 7.0374 - accuracy: 0.7712
Epoch 7/100
5/5 [==============================] - 0s 958us/step - loss: 6.7788 - accuracy: 0.7451
Epoch 8/100
5/5 [==============================] - 0s 1ms/step - loss: 6.5259 - accuracy: 0.7451
Epoch 9/100
5/5 [==============================] - 0s 990us/step - loss: 6.2784 - accuracy: 0.7451
Epoch 10/100
5/5 [==============================] - 0s 1ms/step - loss: 6.0380 - accuracy: 0.7451
Epoch 11/100
5/

In [28]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(
    X_test_scaled,
    y_test,
    verbose = 2
)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2/2 - 0s - loss: 0.5377 - accuracy: 0.8235 - 48ms/epoch - 24ms/step
Loss: 0.5377125144004822, Accuracy: 0.8235294222831726


In [29]:
current_time = datetime.now()
current_time.second

4

In [30]:
# Create dictionary to save the results
results_dict = dict()

results_dict['timestamp'] = current_time
results_dict['runtime'] = time.time() - start_time
results_dict['model_loss'] = model_loss
results_dict['model_accuracy'] = model_accuracy

In [31]:
# Print the model architecture
print(f"Input features: {number_input_features}")
print(f"Hidden Layer 0: {hidden_layer0_activation}, {hidden_layer0_neurons}")

# Populate results_dict with model details
results_dict['input_features'] = number_input_features
results_dict['num_layers'] = best_hyper.values['num_layers']
results_dict['hlayer_0_activation'] = hidden_layer0_activation
results_dict['hlayer_0_neurons'] = hidden_layer0_neurons
results_dict['kernel_regularizer'] = reg_kernel_string

for layer in range(1, total_hidden):
    activation = best_hyper.values[f'activation_layer_{layer}']
    neurons = best_hyper.values[f'units_layer_{layer}']
    print(f"Hidden Layer {layer}: {activation}, {neurons}")
    
    results_dict[f'hlayer_{layer}_activation'] = best_hyper.values[f'activation_layer_{layer}']
    results_dict[f'hlayer_{layer}_neurons'] = best_hyper.values[f'units_layer_{layer}']

results_dict['olayer_neurons'] = output_layer_neurons
results_dict['olayer_activation'] = output_layer_activation

# Populate results_dict with tuning details
results_dict['tuning_max_hidden'] = max_hidden_layers
results_dict['tuning_max_neurons'] = max_num_neurons
results_dict['tuning_step_count'] = step_count
results_dict['activation_functions'] = str(activation_functions)
results_dict['tuning_tuner_epochs'] = tuner_max_epochs
results_dict['tuning_search_epochs'] = search_max_epochs
results_dict['tuning_hp_iterations'] = hp_iterations

# Populate results_dict with compilation details
results_dict['compile_loss'] = compile_loss
results_dict['compile_optimizer'] = compile_opt

Input features: 31
Hidden Layer 0: tanh, 51
Hidden Layer 1: leaky_relu, 61
Hidden Layer 2: leaky_relu, 26


In [32]:
# Change message
change_message = input("Changes from previous iteration: ")

# Append to results_dict
results_dict['change_message'] = change_message

Changes from previous iteration:  no change, removed hardcoding from functions


In [33]:
# Display the dictionary
results_dict

{'timestamp': datetime.datetime(2024, 1, 3, 8, 47, 4, 62841),
 'runtime': 35.98290300369263,
 'model_loss': 0.5377125144004822,
 'model_accuracy': 0.8235294222831726,
 'input_features': 31,
 'num_layers': 2,
 'hlayer_0_activation': 'tanh',
 'hlayer_0_neurons': 51,
 'kernel_regularizer': 'L1(0.010)',
 'hlayer_1_activation': 'leaky_relu',
 'hlayer_1_neurons': 61,
 'hlayer_2_activation': 'leaky_relu',
 'hlayer_2_neurons': 26,
 'olayer_neurons': 1,
 'olayer_activation': 'sigmoid',
 'tuning_max_hidden': 4,
 'tuning_max_neurons': 61,
 'tuning_step_count': 5,
 'activation_functions': "['relu', 'leaky_relu', 'tanh', 'elu', 'selu', 'exponential', 'softmax', 'softplus']",
 'tuning_tuner_epochs': 20,
 'tuning_search_epochs': 20,
 'tuning_hp_iterations': 2,
 'compile_loss': 'binary_crossentropy',
 'compile_optimizer': 'adam',
 'change_message': 'no change, removed hardcoding from functions'}

In [34]:
# Convert the dictionary to a dataframe
results_df = pd.DataFrame(results_dict, index=[0])
results_df.head()

,timestamp,runtime,model_loss,model_accuracy,input_features,num_layers,hlayer_0_activation,hlayer_0_neurons,kernel_regularizer,hlayer_1_activation,...,tuning_max_hidden,tuning_max_neurons,tuning_step_count,activation_functions,tuning_tuner_epochs,tuning_search_epochs,tuning_hp_iterations,compile_loss,compile_optimizer,change_message
0,2024-01-03 08:47:04.062841,35.982903,0.537713,0.823529,31,2,tanh,51,L1(0.010),leaky_relu,...,4,61,5,"['relu', 'leaky_relu', 'tanh', 'elu', 'selu', ...",20,20,2,binary_crossentropy,adam,"no change, removed hardcoding from functions"


In [35]:
# Performance tracker
tracker_path = "../resources/performance_tracker.csv"

# Model percentage
model_pct = round(model_accuracy, 3)

# Check if the CSV exists
if os.path.exists(tracker_path):
    
    # Read the existing CSV
    tracker_df = pd.read_csv(tracker_path)
    
    # Append the new row of data
    updated_df = pd.concat([tracker_df, results_df], ignore_index=True)
    
    # Update the CSV file
    updated_df.to_csv(tracker_path, index=False)
    
    # Export model to HDF5 file
    nn.save(f'../models/run_{len(tracker_df)}_{model_pct}.h5', save_format='h5')

else:    
    # Export to CSV
    results_df.to_csv(tracker_path, index=False)
    
    # Export model to HDF5 file
    nn.save(f'../models/run_0_{model_pct}.h5', save_format='h5')

/Users/alyssahondrade/anaconda3/envs/dev/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
